<a href="https://colab.research.google.com/github/AlexeySto/Data_collection_and_markup_DZ_seminar_9/blob/main/Data_collection_and_markup_DZ_seminar_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание к уроку № 9.<br>
## Инструменты разметки наборов данных

### Задание 1.<br>
Выберите датасет, который имеет отношение к вашей области интересов или исследований. Датасет должен содержать неструктурированные данные, требующие разметки для решения конкретной задачи, например, анализа настроений или распознавания именованных сущностей.

### Задание 2.<br>
Выполните разметку на основе правил (rule-based labeling) на подмножестве выбранного датасета. Разработайте и реализуйте набор правил или условий, которые позволят автоматически присваивать метки данным на основе определенных шаблонов или критериев.

### Задача 3.<br>
Выполните разметку вручную отдельного подмножества выбранного датасета с помощью выбранного вами инструмента разметки.

### Задача 4.<br>
Объедините данные, размеченные вручную, с данными, размеченными на основе правил. Объедините два подмножества размеченных данных в один набор данных, сохранив при этом соответствующую структуру и целостность.

### Задача 5.<br>
Обучите модель машинного обучения, используя объединенный набор размеченных данных. Разделите датасет на обучающий и тестовый наборы и используйте обучающий набор для обучения модели.

### Задача 6.<br>
Оценить эффективность обученной модели на тестовом датасете. Используйте подходящие метрики оценки. Интерпретируйте результаты и проанализируйте эффективность модели в решении задачи разметки.

In [41]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

In [42]:
# Загружаем датасет
data = pd.read_csv('training.csv', engine='python', encoding='utf-8', on_bad_lines='skip')
# Посмотреть первые 5 строк
data.head()


,target,ids,date,flag,user,text
0,4,1563973200,Mon Apr 20 01:07:49 PDT 2009,NO_QUERY,kitdude21,I can haz view? VEDA 20: Early morning upload...
1,0,2191891505,Tue Jun 16 06:09:10 PDT 2009,NO_QUERY,CalumCurry,OMG.. PIMP DOG KID IS LEAVIN CAPETOWN TODAY A...
2,4,2055591774,Sat Jun 06 09:35:52 PDT 2009,NO_QUERY,krispysmith,Never been so happy to be in a church in my en...
3,0,1557934957,Sun Apr 19 06:38:13 PDT 2009,NO_QUERY,KiviKaamos,my bf hate me. i'm so sad
4,4,1677627307,Sat May 02 03:06:39 PDT 2009,NO_QUERY,Dee_Staack,Bored; aha 50th update


In [43]:
# Разделяем датасет на части для разметки вручную и по правилам
data = shuffle(data)
manual_data, rule_based_data = train_test_split(data, train_size=0.005, random_state=42)
manual_data.to_csv("for_manual_marking.csv", index=False)
manual_data

,target,ids,date,flag,user,text
29090,0,2011497459,Tue Jun 02 19:06:47 PDT 2009,NO_QUERY,jimbiosis,@stemcd Whilst I appreciate that Amelia doesn'...
73642,4,2066408800,Sun Jun 07 10:27:56 PDT 2009,NO_QUERY,michellebialy,"@Lulu_Lime go with the latter, you just don't ..."
143478,4,1694552419,Mon May 04 03:08:24 PDT 2009,NO_QUERY,girlj67,@KNUSEE *hugs* listening to EBTG &quot;Each &a...
146788,0,2203970575,Wed Jun 17 00:59:38 PDT 2009,NO_QUERY,malfana,@Brigita09 A boring day ahead of me too
89566,0,1989680442,Mon Jun 01 02:25:22 PDT 2009,NO_QUERY,twicuz34,thinking Esme needs a nap. she's still a baby...
...,...,...,...,...,...,...
158353,4,1880771495,Fri May 22 02:26:49 PDT 2009,NO_QUERY,alwayscrashing,Love and Rockets! BBC 6Music Goth Day is loads...
80091,4,1997910463,Mon Jun 01 17:40:10 PDT 2009,NO_QUERY,chickidee4537,Rory + Gossip Girl = ahhhhh
102800,0,2185030318,Mon Jun 15 16:46:48 PDT 2009,NO_QUERY,chasehardingxxx,@SethApper uh id move there but i dont have a ...
120368,4,2062637502,Sat Jun 06 23:49:48 PDT 2009,NO_QUERY,Dumskull,@ScruffyPanther Morning


In [44]:
# Функция для разметки данных по правилам:
def label_sentiment(text):
    positive_words = ["good", "excellent", "love", "happy", "wonderful"]
    negative_words = ["bad", "I don't like", "sad", "terrible"]

    if any(word in text for word in positive_words):
        return 2
    elif any(word in text for word in negative_words):
        return 1
    else:
        return 0

# Примените эту функцию к подмножеству данных
rule_based_data['sentiment'] = rule_based_data['text'].apply(label_sentiment)
rule_based_data

,target,ids,date,flag,user,text,sentiment
37594,4,1977119810,Sat May 30 20:08:56 PDT 2009,NO_QUERY,jengoddess888,Magic won! Magic won!!!! whooohhaa!! Howard yo...,0
154185,4,1975833062,Sat May 30 16:00:07 PDT 2009,NO_QUERY,disparkys,@henrygrey party at the lake last night ended ...,0
89625,4,1974236875,Sat May 30 12:43:03 PDT 2009,NO_QUERY,gummibunny24,"la la la la la la la, la la la la la la la la....",2
150085,0,2232087598,Thu Jun 18 19:26:04 PDT 2009,NO_QUERY,YourVillain,"@mykesmith Bear, my missing you is turning me ...",0
28309,4,1989628780,Mon Jun 01 02:14:03 PDT 2009,NO_QUERY,DJGerard97,this band is from kildare in ireland. в™« htt...,0
...,...,...,...,...,...,...,...
143771,4,1998980727,Mon Jun 01 19:26:36 PDT 2009,NO_QUERY,kerrinaa,woot! my dad is bringing me a bean cheese and ...,0
133650,0,2197671753,Tue Jun 16 14:58:19 PDT 2009,NO_QUERY,nicollewindwick,I want to go to thurso again now home really ...,0
109532,4,2054381828,Sat Jun 06 07:13:41 PDT 2009,NO_QUERY,honeymatthews,@EbonyIntuition Tiffany! What it do?? You ain...,0
91532,4,2065476440,Sun Jun 07 08:40:28 PDT 2009,NO_QUERY,bryvrgs,@ATLcatalyst thanks for the follow,0


In [45]:
# Загружаем датасет размеченный в ручную
manual_data = pd.read_csv('manual_data.csv', engine='python', encoding='utf-8', on_bad_lines='skip')

manual_data['sentiment'] = manual_data['sentiment'].replace({'Negative': 0, 'Neutral': 1, 'Positive': 2})
manual_data


<ipython-input-45-139eefd4f95b>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  manual_data['sentiment'] = manual_data['sentiment'].replace({'Negative': 0, 'Neutral': 1, 'Positive': 2})


,annotation_id,annotator,created_at,date,flag,id,ids,lead_time,sentiment,target,text,updated_at,user
0,1,1,2024-10-25T19:49:36.997939Z,Tue Jun 02 11:17:55 PDT 2009,NO_QUERY,798,2006527422,7.096,1,0,feeling tired already (actually woke up tired)...,2024-10-25T19:49:36.997939Z,sognix
1,5,1,2024-10-27T16:24:35.897588Z,Sun May 17 02:40:50 PDT 2009,NO_QUERY,799,1824731463,13.592,1,4,"@frombecca yes u should,but add little nuggets...",2024-10-27T16:24:35.897588Z,LukeWMangan
2,4,1,2024-10-27T16:23:40.303248Z,Mon Jun 01 13:40:15 PDT 2009,NO_QUERY,800,1995550626,12.037,1,4,@BlatzLiquor2 oh man...i guess i've been away ...,2024-10-27T16:23:40.303248Z,majastevanovich
3,7,1,2024-10-27T16:25:51.600549Z,Sat Jun 20 18:33:45 PDT 2009,NO_QUERY,801,2260033803,22.742,1,0,up to early yet again.....,2024-10-27T16:25:51.600549Z,bleezyblaze
4,6,1,2024-10-27T16:24:51.560783Z,Sat Jun 20 20:04:53 PDT 2009,NO_QUERY,802,2260984116,10.158,1,0,I would be in the New Dallas Texas Stadium,2024-10-27T16:24:51.561783Z,_AndreaJonas_
...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,793,1,2024-10-28T15:02:31.868232Z,Tue Jun 02 19:51:49 PDT 2009,NO_QUERY,1590,2011972568,40.349,2,4,@hammerheart I have been looking for internshi...,2024-10-28T15:02:31.868232Z,federalcase
793,794,1,2024-10-28T15:02:42.257929Z,Sun Jun 07 13:39:50 PDT 2009,NO_QUERY,1591,2068195394,8.143,0,0,oh this is so sad a 59 yr old man secret mill...,2024-10-28T15:02:42.257929Z,padfootcullen
794,795,1,2024-10-28T15:02:55.736480Z,Sat Jun 06 08:44:19 PDT 2009,NO_QUERY,1592,2055121413,10.705,1,4,Finally got my eyebrows waxed I'm never going ...,2024-10-28T15:02:55.736480Z,AshleyJCherry
795,796,1,2024-10-28T15:03:30.301003Z,Fri Jun 19 06:23:26 PDT 2009,NO_QUERY,1593,2237807287,31.948,1,0,@natybaby and I'm staying with mom dukes still...,2024-10-28T15:03:30.301003Z,Sweet4283


In [46]:
# Объединим размеченные по правилам и в ручную датасеты:
combined_data = pd.concat([manual_data, rule_based_data], ignore_index=True)
combined_data

,annotation_id,annotator,created_at,date,flag,id,ids,lead_time,sentiment,target,text,updated_at,user
0,1.0,1.0,2024-10-25T19:49:36.997939Z,Tue Jun 02 11:17:55 PDT 2009,NO_QUERY,798.0,2006527422,7.096,1,0,feeling tired already (actually woke up tired)...,2024-10-25T19:49:36.997939Z,sognix
1,5.0,1.0,2024-10-27T16:24:35.897588Z,Sun May 17 02:40:50 PDT 2009,NO_QUERY,799.0,1824731463,13.592,1,4,"@frombecca yes u should,but add little nuggets...",2024-10-27T16:24:35.897588Z,LukeWMangan
2,4.0,1.0,2024-10-27T16:23:40.303248Z,Mon Jun 01 13:40:15 PDT 2009,NO_QUERY,800.0,1995550626,12.037,1,4,@BlatzLiquor2 oh man...i guess i've been away ...,2024-10-27T16:23:40.303248Z,majastevanovich
3,7.0,1.0,2024-10-27T16:25:51.600549Z,Sat Jun 20 18:33:45 PDT 2009,NO_QUERY,801.0,2260033803,22.742,1,0,up to early yet again.....,2024-10-27T16:25:51.600549Z,bleezyblaze
4,6.0,1.0,2024-10-27T16:24:51.560783Z,Sat Jun 20 20:04:53 PDT 2009,NO_QUERY,802.0,2260984116,10.158,1,0,I would be in the New Dallas Texas Stadium,2024-10-27T16:24:51.561783Z,_AndreaJonas_
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159491,NaN,NaN,NaN,Mon Jun 01 19:26:36 PDT 2009,NO_QUERY,NaN,1998980727,NaN,0,4,woot! my dad is bringing me a bean cheese and ...,NaN,kerrinaa
159492,NaN,NaN,NaN,Tue Jun 16 14:58:19 PDT 2009,NO_QUERY,NaN,2197671753,NaN,0,0,I want to go to thurso again now home really ...,NaN,nicollewindwick
159493,NaN,NaN,NaN,Sat Jun 06 07:13:41 PDT 2009,NO_QUERY,NaN,2054381828,NaN,0,4,@EbonyIntuition Tiffany! What it do?? You ain...,NaN,honeymatthews
159494,NaN,NaN,NaN,Sun Jun 07 08:40:28 PDT 2009,NO_QUERY,NaN,2065476440,NaN,0,4,@ATLcatalyst thanks for the follow,NaN,bryvrgs


In [48]:
# Разделяем датасет на обучающий и тестовый
train_data, test_data = train_test_split(combined_data, train_size=0.2, random_state=42)

In [49]:
# функция для обучения модели логистической регрессии на размеченных данных
def traun_model(labeled_data):
    # векторизация текстовых данных
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(labeled_data['text'])
    y = labeled_data['sentiment']

    # обучение модели
    model = LogisticRegression(max_iter=200)
    model.fit(X, y)

    return model, vectorizer

#  Обучаем модель
model, vectorizer = traun_model(train_data)

In [50]:
# Оценка модели
X_test = vectorizer.transform(test_data['text'])
y_test_predicted = model.predict(X_test)
f1 = f1_score(test_data['sentiment'], y_test_predicted, average='weighted')

f1

0.9554260389612722